<a href="https://colab.research.google.com/github/JenniferStatter/TensorflowBasics/blob/main/Happy_or_Sad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [95]:
import tensorflow as tf
import os
import zipfile

DESIRED_ACCURACY =0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('acc') > DESIRED_ACCURACY):
      print('\nReached 99.9% accuracy, so cancelling training...')
      self.model.stop_training = True

callbacks = myCallback()

--2021-05-25 03:32:20--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.215.128, 173.194.216.128, 173.194.217.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.215.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2021-05-25 03:32:20 (199 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [96]:
# This code block should create an instance of an ImageDataGenerator called train_datagen
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255.0)

train_generator = train_datagen.flow_from_directory(
    '/tmp/h-or-s/',
    target_size=(150,150),
    batch_size=80,
    class_mode='binary')
# Expected output: 'Found 80 images belonging to two classes'

Found 80 images belonging to 2 classes.


In [97]:
# This code block should call model.fit_generator and train for
# a number of epochs.
history = model.fit_generator(
    train_generator,
    steps_per_epoch=1,
    epochs=25,
    verbose=1,
    callbacks=[callbacks])
# expected output: "Reached 99.9% accuracy so cancelling training!"

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
1/1 [==============================] - 4s 4s/step - loss: 0.6959 - acc: 0.5000
Epoch 2/25
1/1 [==============================] - 3s 3s/step - loss: 0.9196 - acc: 0.5000
Epoch 3/25
1/1 [==============================] - 3s 3s/step - loss: 0.7643 - acc: 0.5000
Epoch 4/25
1/1 [==============================] - 3s 3s/step - loss: 0.6939 - acc: 0.5000
Epoch 5/25
1/1 [==============================] - 3s 3s/step - loss: 0.6064 - acc: 0.5000
Epoch 6/25
1/1 [==============================] - 3s 3s/step - loss: 0.5443 - acc: 0.6625
Epoch 7/25
1/1 [==============================] - 3s 3s/step - loss: 0.4432 - acc: 0.9375
Epoch 8/25
1/1 [==============================] - 3s 3s/step - loss: 0.3368 - acc: 0.9375
Epoch 9/25
1/1 [==============================] - 3s 3s/step - loss: 0.2456 - acc: 0.9375
Epoch 10/25
1/1 [==============================] - 3s 3s/step - loss: 0.2115 - acc: 0.9250
Epoch 11/25
1/1 [==============================] - 3s 3s/step - loss: 0.4168 - acc: 0.7500
Epoch 12

In [ ]:
#import matplotlib.pyplot as plt
#plt.imshow(training_images[0])
#print(training_labels[0])
#print(training_images[0])

In [98]:
#@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
#def next_collatz(x):
  #print("Tracing with", x)
 # return tf.where(x % 2 == 0, x // 2, 3 * x + 1)



In [99]:
classifications = model.predict(train_generator)
print(classifications[0])


[0.9753275]


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is happy")
  else:
    print(fn + " is sad")